In [10]:
import os
import pathlib
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
from functools import partial
import json

import neat
import multiprocess as mp

from features import extract_features_map
from metrics import score
import yulopti
from utils import run_command
from jsonpath_ng import parse

In [20]:
nr_opt_map = {
    0: 'stop',
    1: 'f',
    2: 'l',
    3: 'c',
    4: 'C',
    5: 'U',
    6: 'n',
    7: 'D',
    8: 'E',
    9: 'v',
    10: 'e',
    11: 'j',
    12: 's',
    13: 'x',
    14: 'I',
    15: 'O',
    16: 'o',
    17: 'stop', #'i',
    18: 'g',
    19: 'h',
    20: 'F',
    21: 'T',
    22: 'L',
    23: 'M',
    24: 'm',
    25: 'V',
    26: 'a',
    27: 't',
    28: 'r',
    29: 'p',
    30: 'S',
    31: 'u',
    32: 'd',
    
}

opt_nr_map = {
    'stop': 0,
    'f': 1,
    'l': 2,
    'c': 3,
    'C': 4,
    'U': 5,
    'n': 6,
    'D': 7,
    'E': 8,
    'v': 9,
    'e': 10,
    'j': 11,
    's': 12,
    'x': 13,
    'I': 14,
    'O': 15,
    'o': 16,
    'i': 17,
    'g': 18,
    'h': 19,
    'F': 20,
    'T': 21,
    'L': 22,
    'M': 23,
    'm': 24,
    'V': 25,
    'a': 26,
    't': 27,
    'r': 28,
    'p': 29,
    'S': 30,
    'u': 31,
    'd': 32,
}

In [21]:
def report(msg):
    with open(f"./logs/{os.getpid()}_log.txt", "a") as f:
        f.write(f"{msg}\n")
        print(f"{msg}\n")

In [22]:
max_n = 50
max_rep = 5
max_input_repetition = 5
stop_stats = {
    "features_repetition": 0,
    "steps_repetition": 0,
    "max_steps_repetition": 0,
    "stop_condition": 0,
    "grow_too_much": 0,
}

features_stats = {}
outputs_stats = {}

def eval_genome(net, yul):
    yul_file, yul_file_content = yul
    orig_size = len(yul_file_content)
    last_size = 0
    rep = 0
    rep_index = 0
    input_rep = 0
    last_step = None
    opt_steps = []
    last_feature_vec = None
    last_binary = None
    cached_input_output = {}
    
    # stats_features_repetition = 0
    # stats_steps_repetition = 0
    # stats_max_steps_repetition = 0
    # stats_stop_condition = 0
    
    # begin section
    opt_yul_code = yulopti.steps(yul_file_content, 'hgfo')
    
    # steps section
    for i in range(max_n):
        #print("Features from " + tmp_yul_file + " steps: " + "".join(opt_steps))
        feature_vec = extract_features_map(yul_json=yulopti.to_json(opt_yul_code))
        last_output_vec = [0 for _ in range(33)]
        if last_step:
            last_output_vec[opt_nr_map[last_step]] = 1
        input_vec = feature_vec # + last_output_vec
        
        input_vec_hash = hash(tuple(input_vec))
        # features_stats[input_vec_hash] = input_vec
        # print(f"Process id: {os.getpid()}, feature vec: {input_vec}")
        if input_vec_hash in cached_input_output:
            output = list(cached_input_output[input_vec_hash])
        else:
            output = net.activate(input_vec)
            cached_input_output[input_vec_hash] = output
        best_output_value = max(output)
        potential_steps = [nr_opt_map[idx] for idx, value in enumerate(output) if best_output_value == value]
        
        # if len(opt_steps) > 20:
        #     report(f"Case of long steps ({tuple(opt_steps)}), previous features hash {hash(tuple(last_feature_vec))} \n"
        #            f"Current features hash {hash(tuple(feature_vec))}, last step: {last_step}, potential steps: {potential_steps}\n"
        #            f"rep_index: {rep_index}")
        #     break
            
        if last_feature_vec != feature_vec:
            input_rep = 0
            rep_index = 0
        else:
            input_rep += 1
            if input_rep >= max_input_repetition:
                # print("Break due to features repetition")
                #stats_features_repetition += 1
                stop_stats["features_repetition"] += 1
                break
        
        if rep_index > len(potential_steps) - 1:
            # report(f"Case of reach limit of potential steps ({potential_steps})")
            rep_index = 0
        step = potential_steps[rep_index]
        if last_step == step:
            if last_feature_vec != feature_vec:
                rep += 1
            else:
                if len(potential_steps) > rep_index + 1:
                    rep_index += 1
                    rep = 0
                    step = potential_steps[rep_index]
                else:
                    rep += 1
        else:
            rep = 0
            rep_index = 0
        
        if rep == max_rep:
            # print("Break due to max step repetition")
            # stats_max_steps_repetition += 1
            stop_stats["max_steps_repetition"] += 1
            break
                
        if step == 'stop':
            # print("Break due to stop")
            # stats_stop_condition += 1
            stop_stats["stop_condition"] += 1
            break
        
        last_step = step
        last_feature_vec = feature_vec
        opt_yul_code = yulopti.steps(opt_yul_code, step)
        last_size = len(opt_yul_code)
        opt_steps.append(step)
                
        if last_size > orig_size * 3:
            # print("Break due to stop")
            # stats_stop_condition += 1
            stop_stats["grow_too_much"] += 1
            print("Blowing file " + str(yul_file) + " due to steps " + "".join(opt_steps))
            return 0 # big penalty for this
                
    result = score(yul_file, "".join(opt_steps))
    if result < 0.6:
       print("Fatal score (" + str(result) + ") for " + str(yul_file) + " with steps " + "".join(opt_steps))
    
    # if result > 1.05 or len(opt_steps) > 40:
    #     print(f"opt steps for result {result}: " + "".join(opt_steps))
    #init_steps="fDnTOcmuO"
    
    # if stats_stop_condition > 0 and len(opt_steps) == 0:
    #     print("Stop immediately")
    #print(json.dumps(features_stats))
    #print(json.dumps(outputs_stats))
    # report(f"opt steps for result {result}: " + "".join(opt_steps))
    
    return result
 


In [23]:
samples_dir = pathlib.Path().absolute() / "samples_real"
yul_file_content_map = {}


for yul_file in os.listdir(samples_dir):
    yul_file_path = samples_dir / yul_file
    yul_file_content_map[yul_file_path] = open(yul_file_path, 'r').read()

executor = mp.Pool(12)
#executor = ThreadPoolExecutor(1)

# def create_eval_genomes(samples_dir, aggregate_fn):
#     def eval_genomes(genomes, config):
#         print("Number of genomes: " + str(len(genomes)))
#         for genome_id, genome in genomes:
#             genome.fitness = 0.0
#             net = neat.nn.FeedForwardNetwork.create(genome, config)
#             genome.fitness = []
#             
#             for result in executor.map(partial(eval_genome, net), yul_file_content_map.items()):
#                 # fitness, stats_features_repetition, stats_steps_repetition, stats_max_steps_repetition, stats_stop_condition = result
#                 genome.fitness.append(result)
#                 # stop_stats["features_repetition"] += stats_features_repetition
#                 # stop_stats["steps_repetition"] += stats_steps_repetition
#                 # stop_stats["max_steps_repetition"] += stats_max_steps_repetition
#                 # stop_stats["stop_condition"] += stats_stop_condition
#             aggregated_fitness = aggregate_fn(genome.fitness)
#             #print("Fitness genome " + str(genome_id) + " max: " + str(max(genome.fitness)) + " min: " + str(min(genome.fitness)) + ", fitness: " + str(aggregated_fitness))
#             genome.fitness = aggregated_fitness
#         
#         print(json.dumps(stop_stats))
#         print(run_command("ls -1 ./cache | wc -l"))
#         
#         # with open("stats.txt", "a") as f:
#         #     print(json.dumps(features_stats), file=f)
#         #     print(json.dumps(outputs_stats), file=f)
#         
# 
#     return eval_genomes
def create_eval_genomes(samples_dir, aggregate_fn):
    def eval_genomes(genomes, config):
        print("Number of genomes: " + str(len(genomes)))
        def _eval_genome(item):
            genome_id, genome = item
            genome.fitness = 0.0
            net = neat.nn.FeedForwardNetwork.create(genome, config)
            genome.fitness = []
            
            for result in [eval_genome(net, x) for x in yul_file_content_map.items()]:
                # fitness, stats_features_repetition, stats_steps_repetition, stats_max_steps_repetition, stats_stop_condition = result
                genome.fitness.append(result)
                # stop_stats["features_repetition"] += stats_features_repetition
                # stop_stats["steps_repetition"] += stats_steps_repetition
                # stop_stats["max_steps_repetition"] += stats_max_steps_repetition
                # stop_stats["stop_condition"] += stats_stop_condition
            aggregated_fitness = aggregate_fn(genome.fitness)
            #print("Fitness genome " + str(genome_id) + " max: " + str(max(genome.fitness)) + " min: " + str(min(genome.fitness)) + ", fitness: " + str(aggregated_fitness))
            genome.fitness = aggregated_fitness
            return genome_id, genome
        
        results = {k: v for k,v in executor.map(_eval_genome, genomes)}
        for genome_id, genome in genomes:
            ext_genome = results[genome_id]
            genome.key = ext_genome.key
            genome.connections = ext_genome.connections
            genome.nodes = ext_genome.nodes
            genome.fitness = ext_genome.fitness
        
        print(json.dumps(stop_stats))
        print(run_command("ls -1 ./cache | wc -l"))
        print(yulopti.steps.cache_info())
        
        # with open("stats.txt", "a") as f:
        #     print(json.dumps(features_stats), file=f)
        #     print(json.dumps(outputs_stats), file=f)
        

    return eval_genomes

In [24]:
def run(config_file):
    # Load configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(20))

    # Run for up to 300 generations.
    avg = lambda n: sum(n)/len(n)
    median = lambda n: sorted(n)[len(n) // 2]
    winner = p.run(create_eval_genomes(samples_dir, min), 500)
    os.system('find ./cache -name "*.yul" -delete')

    # Display the winning genome.
    print('\nBest genome:\n{!s}'.format(winner))
    
    return winner
    
# Determine path to configuration file. This path manipulation is
# here so that the script will run successfully regardless of the
# current working directory.
local_dir = './'
config_path = os.path.join(local_dir, 'config-feedforward')
winner = run(config_path)


 ****** Running generation 0 ****** 

Number of genomes: 100
{"features_repetition": 0, "steps_repetition": 0, "max_steps_repetition": 0, "stop_condition": 0, "grow_too_much": 0}
       0

CacheInfo(hits=0, misses=51, maxsize=None, currsize=51)
Population's average fitness: 0.60589 stdev: 0.14065
Best fitness: 0.76286 - size: (33, 660) - species 85 - id 85
Average adjusted fitness: 0.606
Mean genetic distance 3.402, standard deviation 0.332
Population of 200 members in 100 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0     2      0.6    0.622     0
     2    0     2      0.6    0.622     0
     3    0     2      0.7    0.665     0
     4    0     2      0.7    0.659     0
     5    0     2      0.6    0.622     0
     6    0     2      0.6    0.622     0
     7    0     2      0.6    0.622     0
     8    0     2      0.6    0.622     0
     9    0     2      0.7    0.659     0
    10    0     2      0.7    0.659     0
    11  

KeyboardInterrupt: 

In [7]:
winner.fitness

0.8546365914786967

In [8]:
yul_file_content_map.keys()
yul_file_content_map.keys()
yul_file_content_map.keys()
key = pathlib.PosixPath('/Users/krzysztoffonal/programming/pwr/solc-optimizer/samples_test/verifier.yul')
content = open(key, 'r').read()

In [9]:
config_path = os.path.join(local_dir, 'config-feedforward')
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_path)
net = neat.nn.FeedForwardNetwork.create(winner, config)
eval_genome(net, (key, content))

0.8374461417939679

In [ ]:
contract = """
object "Base64_57" {
    code {
        /// @src 0:144:1727  "library Base64 {..."
        mstore(64, memoryguard(128))
        if callvalue() { revert_error_ca66f745a3ce8ff40e2ccaf1ad45db7774001b90d25810abd9040049be7bf4bb() }

        let _1 := allocate_unbounded()
        codecopy(_1, dataoffset("Base64_57_deployed"), datasize("Base64_57_deployed"))

        setimmutable(_1, "library_deploy_address", address())

        return(_1, datasize("Base64_57_deployed"))

        function allocate_unbounded() -> memPtr {
            memPtr := mload(64)
        }

        function revert_error_ca66f745a3ce8ff40e2ccaf1ad45db7774001b90d25810abd9040049be7bf4bb() {
            revert(0, 0)
        }

        /// @src 0:144:1727  "library Base64 {..."
        function constructor_Base64_57() {

            /// @src 0:144:1727  "library Base64 {..."

        }
        /// @src 0:144:1727  "library Base64 {..."

    }
    /// @use-src 0:"data/OptimizorClub.sol"
    object "Base64_57_deployed" {
        code {
            /// @src 0:144:1727  "library Base64 {..."
            mstore(64, memoryguard(128))

            let called_via_delegatecall := iszero(eq(loadimmutable("library_deploy_address"), address()))

            revert_error_42b3090547df1d2001c96683413b8cf91c1b902ef5e3cb8d9f6f304cf7446f74()

            function shift_right_224_unsigned(value) -> newValue {
                newValue :=

                shr(224, value)

            }

            function allocate_unbounded() -> memPtr {
                memPtr := mload(64)
            }

            function revert_error_42b3090547df1d2001c96683413b8cf91c1b902ef5e3cb8d9f6f304cf7446f74() {
                revert(0, 0)
            }

        }

        data ".metadata" hex"a2646970667358221220b9a04b6f4b5cba8084ef122827dbf0782d4d99e4f4bc6e978ce291769d1e29e564736f6c637828302e382e32362d646576656c6f702e323032342e332e32302b636f6d6d69742e61666461363938340059"
    }

}

"""


In [ ]:
print(yulopti.begin(contract))